In [1]:
import logging
import torch

/home/jonhue/projects/jonhue/twitter-sentiment-classification/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from bert import train
from train_test_split import select_train, select_eval, select_train_with_cluster

[nltk_data] Downloading package stopwords to /home/jonhue/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jonhue/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jonhue/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

---

In [5]:
MODEL = 'distilbert-base-uncased' # 'cardiffnlp/twitter-roberta-base-sentiment-latest'
TOKENIZER = 'bert-base-uncased'

EPOCHS = 1
BATCH_SIZE = 32

In [6]:
df_train = select_train()
df_eval = select_eval()

In [7]:
model = train(MODEL, TOKENIZER, device, df_train, df_eval, batch_size=BATCH_SIZE, epochs=EPOCHS)

100%|██████████| 40/40 [00:05<00:00,  7.15ba/s]
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids, index, text. If token_type_ids, index, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/home/jonhue/projects/jonhue/twitter-sentiment-classification/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 160000
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 5000


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Bce,Auc,Confidence,Confidence Std,Correct Confidence,Correct Confidence Std,Incorrect Confidence,Incorrect Confidence Std
1,0.300400,0.290397,0.874175,0.865289,0.886680,0.875854,0.290397,0.947646,1.778359,1.217718,1.940663,1.202755,0.650748,0.529082


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids, index, text. If token_type_ids, index, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40000
  Batch size = 8
INFO:root:---
* accuracy: 0.874175
* precision: 0.8652890145238328
* recall: 0.8866803176347201
* f1: 0.8758540736538318
---
INFO:root:---
* bce: 0.2903966647479203
* auc: 0.9476457382614889
---
Saving model checkpoint to bert_data/test_trainer/checkpoint-5000
Configuration saved in bert_data/test_trainer/checkpoint-5000/config.json
Model weights saved in bert_data/test_trainer/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in bert_data/test_trainer/checkpoint-5000/tokenizer_config.json
Special tokens file saved in bert_data/test_trainer/checkpoint-5000/special_tokens_map.json


Training com

In [8]:
model.save_pretrained('test-x')

Configuration saved in test-baseline/config.json
Model weights saved in test-baseline/pytorch_model.bin
